In [11]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal, interpolate
import copy
import os 
import pandas as pd
os.chdir('/home/kkotzen/research/PPG_sleepstaging/')
from pathlib import Path
from src.parsing.MESAParser import MESAParser
# from src.parsing.utils.comp_ppg_signal_quality import ppg_window_quality, ppg_window_quality_summary, moving_average_filter

def duplicate(x):
    return np.array([[x_]*2 for x_ in x]).flatten()

def hold_line(t, x):
    print(t.shape, x.shape)
    t_ = duplicate(t)[1:-1]
    x_ = duplicate(x)
    return t_, x_

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from scipy import signal, stats

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

def cheby2_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.cheby2(order, 20, [low, high], btype='band')
    return b, a

def cheby2_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = cheby2_bandpass(lowcut, highcut, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

def standardize_signal(data):
    return stats.zscore(data.astype(np.float32)).astype(np.float16)

fs = 256
highcut = 8
lowcut = 0.5
order=4

In [13]:
dl = MESAParser()
patients = ['6632'] #dl.database_all_patient_IDs

ppg = butter_bandpass_filter(dl.load_signal(patients[0], 'Pleth'), lowcut, highcut, 256, order)
ecg = dl.load_signal(patients[0], 'EKG')

ecg_peaks = dl.load_annotation(patients[0], 'EKG', 'epltd0', 'Peaks').astype(int)
valid_ecg_peaks = dl.load_annotation(patients[0], 'EKG', 'rpoint', 'Peaks').astype(int)
ecg_peaks = ecg_peaks[ecg_peaks < valid_ecg_peaks[-1]]
ecg_quality = dl.load_quality(patients[0], 'EKG')

#Load the peaks
Aboy_ppg_peaks = dl.load_annotation(patients[0], 'Pleth', 'Aboy', 'Peaks')
Bishop_ppg_peaks = dl.load_annotation(patients[0], 'Pleth', 'Bishop', 'Peaks')
IMS_peaks = dl.load_annotation(patients[0], 'Pleth', 'IMS', 'Peaks')
AMPD_peaks = dl.load_annotation(patients[0], 'Pleth', 'AMPD', 'Peaks')
pulses_peaks = dl.load_annotation(patients[0], 'Pleth', 'pulses', 'Peaks')
# qppg_onsets = dl.load_annotation(patients[0], 'Pleth', 'qppg', 'Onsets')
co_ppg_peaks = dl.load_annotation(patients[0], 'Pleth', 'co_ppg', 'Peaks')
heartpy_peaks = dl.load_annotation(patients[0], 'Pleth', 'heartpy', 'Peaks')

#Limit them to the reference ECG
Aboy_ppg_peaks = Aboy_ppg_peaks[Aboy_ppg_peaks<ecg_peaks[-1]]
Bishop_ppg_peaks = Bishop_ppg_peaks[Bishop_ppg_peaks<ecg_peaks[-1]]
IMS_peaks = IMS_peaks[IMS_peaks<ecg_peaks[-1]]
AMPD_peaks = AMPD_peaks[AMPD_peaks<ecg_peaks[-1]]
pulses_peaks = pulses_peaks[pulses_peaks<ecg_peaks[-1]]
co_ppg_peaks = co_ppg_peaks[co_ppg_peaks<ecg_peaks[-1]]
heartpy_peaks = heartpy_peaks[heartpy_peaks<ecg_peaks[-1]]


# benchmark = ppg_window_quality(ppg_peaks, ecg_peaks, len(ppg), max_HR_detla=20)

In [14]:
from scipy import signal

def calculate_ptt(ppg_peaks, ecg_peaks, fs=256, max_ptt=0.54, min_ptt=0.38, smoothing_length=600):
    ptt = np.zeros_like(ecg_peaks)
    ptt[:] = (max_ptt*fs+min_ptt*fs)/2

    for i, ecg_peak in enumerate(ecg_peaks):
        try:
            times = ppg_peaks - ecg_peak
            times = times[times > 0][0]
            if (times < max_ptt*fs and times > min_ptt*fs):
                ptt[i] = times
        except:
            if i > 0:
                ptt[i] = ptt[i - 1]

    ptt = signal.filtfilt(np.ones(smoothing_length) / smoothing_length, 1, ptt, padlen=smoothing_length)
    ptt = np.array(ptt[0:len(ecg_peaks)]).astype(int)
    return ptt

def calculate_delayed_ecg(ppg_peaks, ecg_peaks):
    return ecg_peaks + calculate_ptt(ppg_peaks, ecg_peaks)

ptt_filt = calculate_ptt(Bishop_ppg_peaks, ecg_peaks)
# plt.plot(ecg_peaks/256, ptt/256)


# print(f"Searching for peaks that are within {window} samples of the ECG Peak shifted by {pulse_transit_time} samples")

In [15]:
plt.close("all")
cm = 1/2.54  # centimeters in inches
plt.figure(figsize=(7.9*cm, 5*cm))
ax1 = plt.subplot(1,1,1)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
plt.plot(ecg_peaks/(256*60*60), ptt_filt/256, color='blue')
plt.xlabel("Time (Hours)", fontsize=8)
plt.ylabel("Pulse Arrival Time (Seconds)", fontsize=8)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plt.savefig("./notebooks/Images/PTT_Overnight.pdf")

In [25]:
# delayed_ecg_peaks = calculate_delayed_ecg(co_ppg_peaks, ecg_peaks)
cm = 1/2.54
plt.close('all')
f, (ax2, ax1) = plt.subplots(2,1, figsize=(7.9*cm, 4.9*cm), sharex=True, gridspec_kw={'height_ratios': [1, 1]})
ax1.plot(ppg, color='black')
size=3
# ax2.plot(Aboy_ppg_peaks, np.ones(len(Aboy_ppg_peaks))*0.01, 'o', color='black', label='Aboy', markersize=size)
# ax2.plot(Bishop_ppg_peaks, np.ones(len(Bishop_ppg_peaks))*0.02, 'o', color='black', label='Bishop', markersize=size)
# ax2.plot(AMPD_peaks, np.ones(len(AMPD_peaks))*0.02, 'o', color='black', label='AMPD', markersize=size)
ax2.plot(IMS_peaks, np.ones(len(IMS_peaks))*0.01, 'o', color='red', label='APS', markersize=size)
ax2.plot(co_ppg_peaks, np.ones(len(co_ppg_peaks))*0.02, 'o', color='orange', label='CO', markersize=size)
ax2.plot(pulses_peaks, np.ones(len(pulses_peaks))*0.03, 'o', color='green', label='Pulses', markersize=size)
ax2.plot(heartpy_peaks, np.ones(len(heartpy_peaks))*0.04, 'o', color='blue', label='Heartpy', markersize=size)
ax2.plot(delayed_ecg_peaks, np.ones(len(delayed_ecg_peaks))*0, 'x', color='black', label='ECG', markersize=size)
handles, labels = ax2.get_legend_handles_labels()
# ax2.legend(handles[::-1], labels[::-1], loc='upper right', framealpha=1)
labels_ = ["None"]*(len(labels)+1)
labels_[1:] = [L for L in labels]
ax2.set_yticks(np.arange(len(labels_))*0.01)
ax2.set_yticklabels(labels_[::-1])
ax2.tick_params(axis='both', which='major', labelsize=8)
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
# ax2.set_xticks([])
# ax1.plot(qppg_peaks, ppg[qppg_peaks]+0.06, 'o', color='violet', label='QPPG')

# ax1.plot(Aboy_ppg_peaks, ppg[Aboy_ppg_peaks]+0.01, 'o', color='orange', label='Aboy')
# ax1.plot(Bishop_ppg_peaks, ppg[Bishop_ppg_peaks]+0.02, 'o', color='green', label='Bishop')
# ax1.plot(IMS_peaks, ppg[IMS_peaks]+0.03, 'o', color='red', label='IMS')
# ax1.plot(AMPD_peaks, ppg[AMPD_peaks]+0.04, 'o', color='pink', label='AMPD')
# ax1.plot(co_ppg_peaks, ppg[co_ppg_peaks]+0.05, 'o', color='purple', label='Co_PPG')
# ax1.plot(pulses_peaks, ppg[pulses_peaks]+0.06, 'o', color='gold', label='Pulses')
# # ax1.plot(qppg_peaks, ppg[qppg_peaks]+0.06, 'o', color='violet', label='QPPG')
# ax1.plot(delayed_ecg_peaks, ppg[delayed_ecg_peaks], 'x', color='black', label='ECG Peak Forecast')
ax1.set_ylabel('PPG (mV)', fontsize=8)
ax1.set_xlabel('Time (Seconds)', fontsize=8)
ax1.set_yticks([])
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
plt.savefig("./notebooks/Images/benchmark_color.svg")


In [ ]:
def classify_peaks_and_ecg(ppg_peaks, ecg_delayed, window=40, double_peak_window=50):
    validity = np.zeros_like(Aboy_ppg_peaks)
    for i, peak in enumerate(Aboy_ppg_peaks):
        if np.min(np.abs(ecg_delayed - peak)) < window:
            validity[i] = 1

    detected_ecg = np.zeros_like(ecg_delayed)
    for i, ecg_peak in enumerate(ecg_delayed):
        if np.min(np.abs(ecg_peak-Aboy_ppg_peaks)) < window:
            detected_ecg[i] = 1

    double_peaks = np.zeros_like(Aboy_ppg_peaks)
    double_peaks[1:][np.diff(Aboy_ppg_peaks) < double_peak_window] = 1
    
    valid = ppg_peaks[validity==1]
    invalid = ppg_peaks[(validity==0)+(double_peaks==1)] 
    doubles = ppg_peaks[double_peaks==1]
    missed_ecg = ecg_delayed[detected_ecg==0]

    return valid, invalid, doubles, missed_ecg



In [ ]:
valid, invalid, doubles, missed_ecg = classify_peaks_and_ecg(Aboy_ppg_peaks, delayed_ecg_peaks)

plt.close("all")
plt.figure(figsize=(10,5))
plt.plot(ppg)
plt.plot(valid, ppg[valid], 'o', color='green', label='True Peaks')
plt.plot(invalid, ppg[invalid], 'o', color='red', label="False Peaks")
plt.plot(missed_ecg, ppg[missed_ecg], 'o', color='orange', label="Missed ECG Peaks")
plt.plot(doubles, ppg[doubles], 'x', color='black', label="Double Peaks")
plt.plot(delayed_ecg_peaks, ppg[delayed_ecg_peaks], 'x', label='ECG Forecast')
plt.ylabel('PPG Amplitude',fontsize=14)
plt.xlabel("Time (Samples)", fontsize=14)
plt.legend(loc='upper right')

In [ ]:
def f1_from_metrics(validity, ecg_peaks):
    TP = validity.sum()
    FP = len(validity) - TP
    FN = len(ecg_peaks) - TP
    return (TP)/(TP+0.5*(FP+FN))

print("F1 Score: ", f1_from_metrics(valid, ecg_peaks))

In [ ]:
from src.parsing.armand.feature_comp import bsqi

delayed_ecg_peaks = calculate_delayed_ecg(Bishop_ppg_peaks, ecg_peaks)
print(bsqi(delayed_ecg_peaks, Bishop_ppg_peaks, agw=0.1,fs=256))

In [ ]:
print(ppg_window_quality_summary(patients[0], benchmark, ecg_quality, 'Aboy', 'Peaks'))
print(benchmark[benchmark.ecg_quality > 0.85].quality.mean())

print(benchmark[(benchmark.quality > 0.97) & (benchmark.quality < 0.99)].head())

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os 
os.chdir('/home/kkotzen/research/PPG_sleepstaging/')
from pathlib import Path
from scipy.interpolate import interp1d

from src.parsing.utils.comp_ppg_signal_quality import comp_ppg_signal_quality, ppg_window_quality, \
    ppg_window_quality_summary, limit_to_high_quality_peaks_range

from src.parsing.MESAParser import MESAParser
dl = MESAParser()
patients = dl.database_all_patient_IDs[60:70]

def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

ecg = dl.load_signal(patients[0], 'EKG')

ecg_peaks_epltd = dl.load_annotation(patients[0], 'EKG', 'epltd0', 'Peaks').astype(int)
ecg_peaks_rpoint = dl.load_annotation(patients[0], 'EKG', 'rpoint', 'Peaks').astype(int)
ecg_peaks_xqrs = dl.load_annotation(patients[0], 'EKG', 'xqrs', 'Peaks').astype(int)


ecg_quality = dl.load_quality(patients[0], 'EKG')

ecg_peaks_epltd = limit_to_high_quality_peaks_range(ecg_peaks_epltd, ecg_quality)
ecg_peaks_rpoint = limit_to_high_quality_peaks_range(ecg_peaks_rpoint, ecg_quality)
ecg_peaks_xqrs = limit_to_high_quality_peaks_range(ecg_peaks_xqrs, ecg_quality)


plt.close('all')
f, ax = plt.subplots(1,1, figsize=(10,5))
ax.plot(ecg)
ax.plot(ecg_peaks_rpoint, ecg[ecg_peaks_rpoint], 'x')
# ax.plot(ecg_peaks_epltd, ecg[ecg_peaks_epltd], 'o')
ax.plot(ecg_peaks_xqrs, ecg[ecg_peaks_xqrs], '*')

ax.plot(np.arange(0, len(ecg_quality)*30*256, 30*256), ecg_quality)

ax.legend()



In [ ]:
# def nan_helper(y):
#     return np.isnan(y), lambda z: z.nonzero()[0]


# def moving_average_filter(rr, win_samples, percent):
#     b_fir = 1 / (2 * win_samples) * np.append(np.ones(win_samples/2), np.append(0, np.ones(win_samples/2)))
#     print(b_fir)
#     rr_moving_average = signal.filtfilt(b_fir, 1, rr)
#     rr_filtered = rr.copy()
#     rr_filtered[
#         ~((rr < (1 + percent / 100) * rr_moving_average) & (rr > (1 - percent / 100) * rr_moving_average))] = np.nan
#     nans, x = nan_helper(rr_filtered)
#     rr_filtered[nans] = np.interp(x(nans), x(~nans), rr_filtered[~nans])
#     return rr_filtered

In [ ]:
s = np.ones(100)
s[20] = 1.5
z = moving_average_filter(s, 10, 20)

print(s)
print(z)


In [ ]:
import pyedflib
import numpy as np
from pathlib import Path
import os
import tqdm
from joblib import Parallel, delayed

# # file_name = "/home/kkotzen/databases/mesa/polysomnography/edfs/mesa-sleep-0028.edf"
# # f = pyedflib.EdfReader(file_name)
# n = f.signals_in_file
# signal_labels = f.getSignalLabels()
# print([e for e in enumerate(signal_labels)])
# x = f.readSignal(22)
# # sigbufs = np.zeros((n, f.getNSamples()[0]))
# # for i in np.arange(n):
# #         sigbufs[i, :] = f.readSignal(i)

source = Path("/home/kkotzen/databases/mesa/polysomnography/npys/Pleth/")
dest = Path("/home/kkotzen/databases/mesa/polysomnography/csvs/Pleth")

def process(f):
    x = np.load(source / f)
    filepath = dest / f"{f[:-3]}csv"
    np.savetxt(filepath, x, delimiter=",")

patients = ['0028']#, '0050','0054','0081','0388','0445','0485','0536','0548','0554','0648','0702','0823','0854','0873','1090','1474','1569','1570','1595','1619','1655','1732','1781','1842','1876','2129','2163','2183','2251','2264','2504','2780','2811','2846','2906','3112','3158','3510','3529','3612','3630','3761','3897','4000','4014','4032','4152','4168','4186','4461','4477','4488','4861','5118','5151','5169','5203','5231','5443','5555','5788','5939','5955','6005','6280','6314','6376','6430','6583','6723','6726']
print(os.listdir(source)[0][11:15])
files = [f for f in os.listdir(source) if f[11:15] in patients]
Parallel(n_jobs=8)(delayed(process)(f) for f in tqdm.tqdm(files))

# for f in tqdm.tqdm(os.listdir(source)):
#     process(f)


In [ ]:
# 3495
co_ppg = "/home/kkotzen/databases/mesa/polysomnography/annotations/Pleth/co_ppg/Peaks/mesa-sleep-0050-annotations.npy"
aboy = "/home/kkotzen/databases/mesa/polysomnography/annotations/Pleth/Aboy/Peaks/mesa-sleep-0050-annotations.csv"

co_ppg_peaks = np.load(co_ppg)
aboy_peaks = np.loadtxt(aboy).astype(int)

signal = np.load("/home/kkotzen/databases/mesa/polysomnography/npys/Pleth/mesa-sleep-0050-Pleth.npy")

In [ ]:
plt.close('all')
plt.plot(signal)
plt.plot(co_ppg_peaks, signal[co_ppg_peaks], 'x')
plt.plot(aboy_peaks, signal[aboy_peaks], 'o')


In [ ]:

def duplicate(x):
    return np.array([[x_]*2 for x_ in x]).flatten()

def hold_line(t, x):
    t = [i for i in t]
    x = [i for i in x]
    
    if len(t) - len(x) == 0:
            t.append(t[-1]+t[-1]-t[-2])
    if len(t)-len(x) != 1:
            raise ValueError()
            
    t_ = duplicate(t)[1:-1]
    x_ = duplicate(x)
    return t_, x_

X = np.array([0.25, 0.5, 0.25, 0.75, 0.25, 1])
T = np.array([0,30,60,90,120, 150])

t, x = hold_line(T, X)
plt.close('all')
plt.plot(t, x)